In [ ]:
using JuMP
mod =  Model()
@variable(mod, x >= 0)
@expression(mod, costs, 2*x)

ErrorException: Expected mod to be a JuMP model, but it has type typeof(mod)

In [41]:
m = Model()
@variable(m, P[N])
@constraint(m, c[n in N], P[n] <= NODE[n].max)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:2
And data, a 2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 c[1] : P[1] ≤ 4
 c[2] : P[2] ≤ 5

In [1]:
using CSV, DataFrames, NamedArrays

function to_Structs(structure::DataType, inputs_dir:: String, filename:: String):: Vector{structure}
    file_dir = joinpath(inputs_dir, filename)
    struct_names = fieldnames(structure)
    struct_types = fieldtypes(structure)

    first_csvlines = CSV.File(joinpath(inputs_dir, filename); limit=1)
    csv_header = Tuple(propertynames(first_csvlines))

    @assert csv_header == struct_names """Column names of $filename does not match the fields of the structure $structure."""

    df = CSV.read(file_dir, DataFrame; types=Dict(zip(struct_names, struct_types)))
    cols = Tuple(df[!, col] for col in names(df))
    V = structure.(cols...)    
    
    return V
end

function to_multidim_NamedArray(structures:: Vector{T}, dims:: Vector{Symbol}, value:: Symbol):: NamedArray{Union{Missing, Float64}} where {T} 
  
    vals_in_dim = [unique(getfield.(structures, d)) for (i, d) in enumerate(dims)]
    
    arr = Array{Union{Missing, Float64}}(missing, length.(vals_in_dim)...)
    arr = NamedArray(arr, vals_in_dim, dims)

    for s in structures
        arr[getfield.(Ref(s), dims)...] = getfield(s, value)
    end

    return arr
end

to_multidim_NamedArray (generic function with 1 method)

In [2]:
struct Bus
    bus_id:: String
    kv:: Float64
    type:: String
    lat:: Float64
    lon:: Float64
    slack:: Bool
end


In [3]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "buses.csv"

    # Get a list of Bus structures
    buses = to_Structs(Bus, inputs_dir, "buses.csv")

    # Get a list of the bus IDs
    N = getfield.(buses, :bus_id)

    # Transform buses into NamedArray, so we can access buses by their IDs
    buses = NamedArray(buses, (N))

    # Get slack bus
    slack_bus = buses[ findfirst([n.slack == true for n in buses]) ].bus_id
    

"lima"

In [4]:
struct Load
    bus_id:: String
    sc_id:: String
    t_id:: String
    load:: Float64
end
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "loads.csv"

LOADS = to_Structs(Load, inputs_dir, filename)
arr = to_multidim_NamedArray(LOADS, [:bus_id, :sc_id, :t_id], :load)

2×2×3 Named Array{Union{Missing, Float64}, 3}

[:, :, t_id=1] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     4.0  missing
lima           │ missing      2.0

[:, :, t_id=2] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     5.0  missing
lima           │ missing      1.0

[:, :, t_id=3] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     6.0  missing
lima           │ missing      4.0

In [13]:
struct CapacityFactor
    gen_id:: String
    tp_id:: String
    sc_id:: String
    capacity_factor:: Float64
end

In [36]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "capacity_factors.csv"
CF = to_Structs(CapacityFactor, inputs_dir, filename)
cf = to_multidim_NamedArray(CF, [:gen_id, :tp_id, :sc_id], :capacity_factor)

2×3×2 Named Array{Union{Missing, Float64}, 3}

[:, :, sc_id=low] =
gen_id ╲ tp_id │   1    2    3
───────────────┼──────────────
sun_lim        │ 0.1  0.2  3.0
winds          │ 0.5  1.0  0.2

[:, :, sc_id=high] =
gen_id ╲ tp_id │   1    2    3
───────────────┼──────────────
sun_lim        │ 0.1  0.2  3.0
winds          │ 0.5  0.2  1.0

In [8]:
S = LOADS

dims = [:bus_id, :sc_id, :t_id]
val = :load

vals_in_dim = [unique(getfield.(S, d)) for (i, d) in enumerate(dims)]

arr = Array{Union{Missing, Float64}}(missing, length.(vals_in_dim)...)
arr = NamedArray(arr, vals_in_dim, dims)
for s in S
    arr[getfield.(Ref(s), dims)...] = getfield(s, val)
end
arr

2×2×3 Named Array{Union{Missing, Float64}, 3}

[:, :, t_id=1] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     4.0  missing
lima           │ missing      2.0

[:, :, t_id=2] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     5.0  missing
lima           │ missing      1.0

[:, :, t_id=3] =
bus_id ╲ sc_id │     low     high
───────────────┼─────────────────
sd             │     6.0  missing
lima           │ missing      4.0

In [10]:
arr["sd", "low", 1]

4.0

In [6]:
struct Generator
    gen_id:: String
    gen_tech:: String
    bus_id:: String
    c2:: Float64
    c1:: Float64
    c0:: Float64
    invest_cost:: Float64
    exist_cap:: Float64
    cap_limit:: Float64
    var_om_cost:: Float64
end

struct Storage 
    sto_id:: String
    sto_tech:: String
    bus_id:: String
    invest_cost:: Float64
    exist_power_cap:: Float64
    exist_energy_cap:: Float64
    var_om_cost:: Float64
    efficiency:: Float64
    duration:: Float64
end

In [7]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "generators.csv"
gens = to_Structs(Generator, inputs_dir, filename)
G = getfield.(gens, :gen_id)
gens = NamedArray(gens, (G))

3-element Named Vector{Generator}
A       │ 
────────┼──────────────────────────────────────────────────────────────────────────
sun_lim │ Generator("sun_lim", "solar", "lima", 1.0, 2.0, 0.0, 1.0, 1.0, 20.0, 0.5)
winds   │    Generator("winds", "wind", "lima", 2.0, 3.0, 4.0, 2.0, 0.0, 20.0, 0.5)
dirt    │       Generator("dirt", "coal", "sd", 2.0, 3.0, 4.0, 3.0, 0.0, 20.0, 2.0)

In [17]:
GENS_AT_BUS = [ gens[ findall([g.bus_id == n for g in gens]) ] for n in N ]


FieldError: FieldError: type NamedArray has no field `bus_id`, available fields: `array`, `dicts`, `dimnames`

In [45]:
x = [ gens[ findall([g.bus_id == n for g in gens]) ] for n in N ]



2-element Vector{NamedVector{Generator, Vector{Generator}, Tuple{OrderedCollections.OrderedDict{String, Int64}}}}:
 Generator[Generator("dirt", "coal", "sd", 2.0, 3.0, 4.0, 3.0, 0.0, 20.0, 2.0)]
 Generator[Generator("sun_lim", "solar", "lima", 1.0, 2.0, 0.0, 1.0, 1.0, 20.0, 0.5), Generator("winds", "wind", "lima", 2.0, 3.0, 4.0, 2.0, 0.0, 20.0, 0.5)]

In [64]:
G_AT_BUS = NamedArray([ [] for n in N], (N))
for n in N
    G_AT_BUS[n] = names(gens[ findall(g -> g.bus_id == n, gens) ], 1)
end

In [67]:
G_AT_BUS = NamedArray( [ names(gens[ findall(g -> g.bus_id == n, gens) ], 1) for n in N ], (N) )

2-element Named Vector{Vector{String}}
A    │ 
─────┼─────────────────────
sd   │             ["dirt"]
lima │ ["sun_lim", "winds"]

In [21]:
G_AT_BUS = NamedArray( [ names(gens[ findall([g.bus_id == n for g in gens])  ], 1) for n in N ], (N), "bus_id" )

2-element Named Vector{Vector{String}}
bus_id  │ 
────────┼─────────────────────
sd      │             ["dirt"]
lima    │ ["sun_lim", "winds"]

In [20]:
G_AT_BUS = [filter(g -> g.bus_id == n, gens) for n in N]

2-element Vector{NamedVector{Generator, Vector{Generator}, Tuple{OrderedCollections.OrderedDict{String, Int64}}}}:
 Generator[Generator("dirt", "coal", "sd", 2.0, 3.0, 4.0, 3.0, 0.0, 20.0, 2.0)]
 Generator[Generator("sun_lim", "solar", "lima", 1.0, 2.0, 0.0, 1.0, 1.0, 20.0, 0.5), Generator("winds", "wind", "lima", 2.0, 3.0, 4.0, 2.0, 0.0, 20.0, 0.5)]

In [18]:
x = filter(g -> g.bus_id == "lima", gens) 
[i.bus_id for i in x]

2-element Vector{String}:
 "lima"
 "lima"

In [26]:
[ [g.bus_id for g in filter(g -> g.bus_id == n, gens) ] for n in N ]

2-element Vector{Vector{String}}:
 ["sd"]
 ["lima", "lima"]

In [78]:
#using BenchmarkTools
#@benchmark begin
G_AT_BUS = NamedArray( [getfield.(filter(g -> g.bus_id == n, gens).array, :gen_id) for n in N], (N), :bus_id )
GV = ["sun_lim", "winds"]

GV_AT_BUS = intersect.(G_AT_BUS, GV)
#end

2-element Named Vector{Vector{Any}}
bus_id  │ 
────────┼──────
sd      │ Any[]
lima    │ Any[]

In [79]:
G_AT_BUS

2-element Named Vector{Vector{String}}
bus_id  │ 
────────┼─────────────────────
sd      │             ["dirt"]
lima    │ ["sun_lim", "winds"]

In [87]:
fill(GV, 3)

3-element Vector{Vector{String}}:
 ["sun_lim", "winds"]
 ["sun_lim", "winds"]
 ["sun_lim", "winds"]

In [89]:
intersect.(G_AT_BUS, fill(GV, length(N)))

2-element Named Vector{Vector{String}}
bus_id  │ 
────────┼─────────────────────
sd      │             String[]
lima    │ ["sun_lim", "winds"]

In [ ]:
gens_at_bus = NamedArray([(gens[getfield.(gens.array, :bus_id) .== n]).array for n in N], (N), "bus_id")
G at bus = 

2-element Named Vector{Vector{Generator}}
bus_id  │ 
────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
sd      │                                                                               Generator[Generator("dirt", "coal", "sd", 2.0, 3.0, 4.0, 3.0, 0.0, 20.0, 2.0)]
lima    │ Generator[Generator("sun_lim", "solar", "lima", 1.0, 2.0, 0.0, 1.0, 1.0, 20.0, 0.5), Generator("winds", "wind", "lima", 2.0, 3.0, 4.0, 2.0, 0.0, 20.0, 0.5)]

In [54]:
@benchmark begin
G_AT_BUS = NamedArray( [ names(gens[ findall([g.bus_id == n for g in gens])  ], 1) for n in N ], (N), "bus_id" )
end

BenchmarkTools.Trial: 10000 samples with 8 evaluations per sample.
 Range (min … max):  3.380 μs … 768.115 μs  ┊ GC (min … max): 0.00% … 99.01%
 Time  (median):     3.734 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.808 μs ±   7.646 μs  ┊ GC (mean ± σ):  2.00% ±  0.99%

                    ▆▇▇█▁                                      
  ▁▁▃▃▄▄▄▆▅▄▃▃▂▂▂▃▆▇█████▇▄▃▃▂▃▂▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  3.38 μs         Histogram: frequency by time        4.43 μs <

 Memory estimate: 3.84 KiB, allocs estimate: 100.

In [36]:
y = map(p -> p.gen_id, x[2])

2-element Named Vector{String}
A       │ 
────────┼──────────
sun_lim │ "sun_lim"
winds   │   "winds"

In [20]:
using CSV, DataFrames

file = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs/scenarios.csv"
df = CSV.read(file, DataFrame)
cols = Tuple(df[!, col] for col in names(df))
S = Scenario.(cols...)


2-element Vector{Scenario}:
 Scenario(1, "low", 0.6)
 Scenario(2, "high", 0.4)

In [19]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "scenarios.csv"
SX = to_vec_of_structs(Scenario, inputs_dir, filename)

2-element Vector{Scenario}:
 Scenario(1, "low", 0.6)
 Scenario(2, "high", 0.4)

In [50]:
struct Load
    bus_id:: Int64
    sc_id:: Int64
    t_id:: Int64
    value:: Float64
end
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "loads.csv"
L = to_vec_of_structs(Load, inputs_dir, filename)

In [84]:
inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "loads.csv"
load = CSV.read(joinpath(inputs_dir, "loads.csv"),DataFrame, 
types=[Int64, Int64, Int64, Float64]);
load

to_Structs (generic function with 2 methods)

In [88]:
using AxisKeys
x = wrapdims(load, :value, :bus_id, :scenario, :t_id)

(:bus_id, :sc_id, :t_id, :load)

In [ ]:
using CSV, DataFrames, NamedArrays

inputs_dir = "/Users/paul/Documents/CATSExpand/examples/toy_example1/inputs"
filename = "loads.csv"

struct Load
    bus_id:: String
    sc_id:: String
    t_id:: String
    load:: Float64
end


#V = to_Structs(Load, inputs_dir, filename)

MethodError: MethodError: objects of type NTuple{4, Symbol} are not callable
The object of type `NTuple{4, Symbol}` exists, but no method is defined for this combination of argument types when trying to treat it as a callable object.

In [90]:
struct Line
    line_id:: String
    from_bus:: String
    to_bus:: String
    rate:: Float64
    r:: Float64
    x:: Float64
    g:: Float64
    b:: Float64
end

lines = to_Structs(Line, inputs_dir, "lines.csv")

1-element Vector{Line}:
 Line("bart", "lima", "sd", 100.0, 0.01, 0.05, 0.1, 0.05)

In [101]:
function build_admittance_matrix(N:: Vector{String}, lines; include_shunts=false) :: NamedArray{ComplexF64}

# Define admittance matrix (actually it is NamedArray)
# Note: we opt to use a NamedArray so N does not have to be a vector of numbers
#       then, the user has more flexibility to access the admittance matrix, for example, Y["sandiego", "lima"]
num_buses = length(N)

Y =  NamedArray( zeros(Complex, num_buses, num_buses), (N, N), (:bus_id, :bus_id))

for line in lines
# Calculate branch admittance
z = complex(line.r, line.x)
y = 1.0 / z

# Extract from_bus and to_bus from line instance
from_bus = line.from_bus
to_bus = line.to_bus

# Off-diagonal elements. Y_ij = -y_ij
Y[from_bus, to_bus] -= y
Y[to_bus, from_bus] -= y

# Diagonal elements. Note: Y_ii = y_1i + y2i + ... + yii + ...
Y[from_bus, from_bus] += y
Y[to_bus, to_bus] += y
end

if include_shunts
for line in lines
# Calculate shunt admittance 
y_shunt = complex(line.g, line.b)

# Extract bus 
from_bus = line.from_bus
to_bus = line.to_bus

# Add shunt admittance to the current admittance matrix
Y[from_bus, from_bus] += y_shunt
Y[to_bus, to_bus] += y_shunt
end
end

return Y
end


build_admittance_matrix (generic function with 4 methods)

In [102]:
build_admittance_matrix(N, lines)

2×2 Named Matrix{ComplexF64}
bus_id ╲ bus_id │                 sd                lima
────────────────┼───────────────────────────────────────
sd              │  3.84615-19.2308im  -3.84615+19.2308im
lima            │ -3.84615+19.2308im   3.84615-19.2308im

In [107]:
function get_maxFlow(N:: Vector{String}, lines):: NamedArray{Float64}

    num_buses = length(N)
    maxFlow =  NamedArray( zeros(Float64, num_buses), (N), :bus_id )

    for line in lines

        # Extract from_bus and to_bus from line instance
        from_bus = line.from_bus
        to_bus = line.to_bus
        rate = line.rate
        
        maxFlow[from_bus] += rate
        maxFlow[to_bus] += rate

    end
    return maxFlow
end

get_maxFlow (generic function with 1 method)

In [108]:
get_maxFlow(N, lines)

2-element Named Vector{Float64}
bus_id  │ 
────────┼──────
sd      │ 100.0
lima    │ 100.0